In [1]:
%pip install protobuf
import transformers
from transformers import pipeline
from transformers import BertTokenizerFast, BertForQuestionAnswering, TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import pandas as pd
import numpy as np
import torch
from bs4 import BeautifulSoup
from datasets import Dataset
from nltk.tokenize import word_tokenize

print(torch.backends.mps.is_available())

import sys
sys.path.append("../CODE-Baseline")  
import warnings
warnings.filterwarnings('ignore')


from salary_baseline import extract_salary_with_inference


file_path = '../DATASETS/salary_labelled_development_set.csv'
test_file_path = '../DATASETS/salary_labelled_test_set.csv'
ignore_id_path = '../DATASETS/err_salary_develpment.csv'

df = pd.read_csv(file_path)
df_ignore = pd.read_csv(ignore_id_path)
# some y_true is impossible
df = df[~df['job_id'].isin(df_ignore['job_id'])]


model_name = "google/mt5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# qa_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=-1)

Note: you may need to restart the kernel to use updated packages.
True
Development dataset:
Precision: 0.8357
Recall: 0.9524
F1 Score: 0.8902
Accuracy: 0.8889

Test dataset:
Precision: 0.7412
Recall: 0.9206
F1 Score: 0.8212
Accuracy: 0.8219


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
# 清理 HTML 标签
def clean_html_tags(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    for tag in soup(["script", "style"]):
        tag.decompose()
    return soup.get_text(separator="\n", strip=True)

# 构造输入输出
def row_to_input_output(row):
    context = clean_html_tags(f"{row['job_title']} {row['job_ad_details']}")
    prompt = (
        "Extract the salary info from the job ad below and return it as: "
        "\"MinimumSalary MaximumSalary Currency PayPeriod\"\n\n"
    )
    input_text = prompt + context
    output_text = str(row["y_true"]).strip().replace("-", " ")
    return {"input": input_text, "output": output_text}

above512=0
ll=0
tt=1
# 预处理函数
def preprocess(example):
    global above512
    global ll
    global tt
    input_tokens = tokenizer.tokenize(example["input"])
    if len(input_tokens) >=2700 and tt:
      print(example["input"])
      tt=0
    if len(input_tokens) >=512:
      above512+=1
      if len(input_tokens) > ll:
        ll = len(input_tokens)
    model_inputs = tokenizer(example["input"], max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["output"], max_length=64, truncation=True, padding="max_length")["input_ids"]
    labels = [token if token != tokenizer.pad_token_id else -100 for token in labels]
    model_inputs["labels"] = labels
    return model_inputs
  
# tokenized_dataset = squad_dataset
squad_dataset = Dataset.from_list([row_to_input_output(row) for _, row in df.iterrows()])
tokenized_dataset = squad_dataset.map(preprocess)
final_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
print("training data size：", len(final_dataset['train']))
print("testing data size：", len(final_dataset['test']))

print(above512)
print(ll)

Map:   0%|          | 0/2133 [00:00<?, ? examples/s]

Extract the salary info from the job ad below and return it as: "MinimumSalary MaximumSalary Currency PayPeriod"

Principal Advisors - Renewable Energy Projects
Principal Advisors, Renewable Energy Projects
Excellent opportunity to join the Renewable Energy Projects team
Hub based location, Perth, Brisbane, or Montreal
Core new Renewable positions where your expertise can shine
Rio Tinto has recently launched a dedicated team to deliver large renewable energy solutions across our major jurisdictions. Rio Tinto Energy Development is focusing on accelerating the development of future energy assets and securing high quality renewable energy development optionality. The team's focus on the provision of green energy will enable the decarbonisation of the company’s existing industrial operations and support Rio Tinto’s future growth
About the role
RTED has been established to support and accelerate the delivery of the Rio Tinto strategy. RTED is will drive project delivery, with multiple ren